# XGBoost – Explicación Matemática Paso a Paso

XGBoost es una versión optimizada y regularizada del algoritmo de Gradient Boosting. A diferencia del boosting clásico, utiliza **expansión de Taylor de segundo orden** y una función objetivo regularizada para mejorar la eficiencia y evitar overfitting.

---

## 1. Función objetivo de XGBoost

## XGBoost como modelo aditivo

XGBoost es un modelo **aditivo de árboles**, lo que significa que construye el modelo final sumando árboles uno a uno:

$$
\hat{y}_i^{(t)} = \hat{y}_i^{(t-1)} + f_t(x_i)
$$

Donde:
- $\hat{y}_i^{(t)}$ es la predicción del ejemplo $i$ en la iteración $t$
- $f_t(x_i)$ es el nuevo árbol que se entrena en la iteración $t$

---

## Objetivo en cada iteración

En cada paso, el nuevo árbol $f_t$ busca **minimizar la contribución marginal** a la función de pérdida total. Es decir, tomamos la función objetivo y la separamos en:

$$
\mathcal{L}^{(t)} =
\underbrace{
\sum_{i=1}^n l(y_i, \hat{y}_i^{(t-1)}) + \sum_{k=1}^{t-1} \Omega(f_k)
}_{\text{Parte ya construida (constante en esta iteración)}} +
\underbrace{
\sum_{i=1}^n \left[ l(y_i, \hat{y}_i^{(t-1)} + f_t(x_i)) - l(y_i, \hat{y}_i^{(t-1)}) \right] + \Omega(f_t)
}_{\text{Lo que añade el nuevo árbol $f_t$}}
$$

Esto enfatiza que:

- Solo necesitamos **optimizar el segundo término**, ya que el primero no cambia en esta iteración.
- El nuevo árbol $f_t$ se entrena para **mejorar la predicción actual** $\hat{y}_i^{(t-1)}$ sumando una corrección.

Sin segmentar: 

$$
\mathcal{L}^{(t)} =
\sum_{i=1}^n l(y_i, \hat{y}_i^{(t-1)} + f_t(x_i)) + \sum_{k=1}^{t} \Omega(f_k)
$$


---


## ⚙️ Componentes de la función objetivo

- $l(y_i, \hat{y}_i)$: función de pérdida (por ejemplo, log-loss o MSE)
- $\hat{y}_i^{(t)} = \hat{y}_i^{(t-1)} + f_t(x_i)$: predicción en el paso $t$
- $\Omega(f_k)$: penalización por la complejidad del árbol $f_k$

La regularización $\Omega(f)$ se define como:

$$
\Omega(f) = \gamma T + \frac{1}{2} \lambda \sum_{j=1}^{T} w_j^2
$$

- $T$: número de hojas en el árbol
- $w_j$: valor predicho por la hoja $j$
- $\gamma$: penalización por cada hoja (controla el crecimiento del árbol)
- $\lambda$: penalización L2 sobre los valores de las hojas (controla magnitudes extremas)


---
## 2. Expansión de Taylor

### Recordatorio rápido

Cuando una función es complicada, podemos aproximarla cerca de un punto usando una **expansión de Taylor de segundo orden**:

$$
f(x) \approx f(a) + f'(a)(x - a) + \frac{1}{2} f''(a)(x - a)^2
$$

En particular, si centramos la expansión en $a = 0$ (como hace XGBoost), tenemos:

$$
f(x) \approx f(0) + f'(0)x + \frac{1}{2} f''(0)x^2
$$

Esto nos da una **aproximación cuadrática** que es más informativa que una línea recta (como en el gradiente descendente clásico), y aún así es eficiente de optimizar.

---

### Aplicación en XGBoost

Para poder usar boosting con funciones de pérdida arbitrarias (como log-loss o MAE), XGBoost necesita una forma de **aproximar la pérdida** alrededor de las predicciones actuales $\hat{y}_i^{(t-1)}$.

Ahí es donde entra la expansión de Taylor:

$$
l(y_i, \hat{y}_i^{(t)}) = l(y_i, \hat{y}_i^{(t-1)} + f_t(x_i)) \approx l(y_i, \hat{y}_i^{(t-1)}) + g_i f_t(x_i) + \frac{1}{2} h_i f_t(x_i)^2
$$

Donde:
- $g_i = \left.\frac{\partial l}{\partial \hat{y}_i}\right|_{\hat{y}_i^{(t-1)}}$: gradiente
- $h_i = \left.\frac{\partial^2 l}{\partial \hat{y}_i^2}\right|_{\hat{y}_i^{(t-1)}}$: hessiano
- $f_t(x_i)$ es **la predicción del nuevo árbol**, es decir, la corrección que se le va a sumar a la predicción actual

El primer término $l(y_i, \hat{y}_i^{(t-1)})$ se puede ignorar en esta iteración, ya que es constante.

---

#### 🧠 Conexión clave: ¿por qué aparece $f_t(x_i)$?

Recuerda que en la expansión de Taylor:

$$
f(x) \approx f(a) + f'(a)(x - a) + \frac{1}{2} f''(a)(x - a)^2
$$

Si definimos:
- $x = \hat{y}_i^{(t)}$
- $a = \hat{y}_i^{(t-1)}$
- Entonces: $x - a = \hat{y}_i^{(t)} - \hat{y}_i^{(t-1)} = f_t(x_i)$

> ✅ Así que en XGBoost, **$f_t(x_i)$ es exactamente el $(x - a)$ de la expansión de Taylor**.

---

### Resultado

La función que efectivamente se optimiza en cada iteración es:

$$
\tilde{\mathcal{L}}^{(t)} = \sum_{i=1}^n \left[ g_i f_t(x_i) + \frac{1}{2} h_i f_t(x_i)^2 \right] + \Omega(f_t)
$$

Esto convierte la optimización en un problema **cuadrático respecto a $f_t(x_i)$**, lo que permite construir el nuevo árbol de forma eficiente.

---

## ¿Por qué usamos Taylor?

La pérdida original puede ser arbitraria (por ejemplo, log-loss o MAE), y no tiene una forma cerrada fácil de minimizar al agregar un nuevo árbol.  
Por eso, usamos una **expansión de segundo orden** para aproximarla por una función cuadrática, que sí podemos minimizar fácilmente al construir un árbol hoja por hoja.

---

## 3. Árboles como funciones pieza por pieza

Cada nuevo árbol $f_t$ asigna un valor constante $w_j$ a cada región $R_j$ (una hoja). Entonces:

$$
f_t(x_i) = w_j \quad \text{si } x_i \in R_j
$$

Sustituyendo esto en la función objetivo aproximada:

$$
\mathcal{L}^{(t)} \approx \sum_{j=1}^{T} \left[
\sum_{i \in R_j} g_i w_j + \frac{1}{2} \sum_{i \in R_j} h_i w_j^2
\right] + \gamma T + \frac{1}{2} \lambda \sum_{j=1}^{T} w_j^2
$$

Agrupando:

$$
\mathcal{L}^{(t)} \approx \sum_{j=1}^{T} \left[
G_j w_j + \frac{1}{2} (H_j + \lambda) w_j^2
\right] + \gamma T
$$

Con:
- $G_j = \sum_{i \in R_j} g_i$
- $H_j = \sum_{i \in R_j} h_i$


---

## 4. Output Value por hoja

Minimizamos el término por hoja respecto a $w_j$:

$$
\frac{d}{dw_j} \left( G_j w_j + \frac{1}{2} (H_j + \lambda) w_j^2 \right) = 0
$$

Solución:

$$
\textbf{Output value} = w_j^* = -\frac{G_j}{H_j + \lambda}
$$

Nota: esta solución es equivalente al paso de Newton-Raphson aplicado a cada hoja, ya que minimizamos una aproximación cuadrática local de la pérdida.


---
## 5. Similarity Score (calidad de una hoja)

No es una loss en el sentido habitual, pero sí es proporcional a la pérdida que se reduce si mantienes esa hoja y usas el output value óptimo.

Recordemos que la función de pérdida regularizada en una hoja $R_j$ es:

$$
\mathcal{L}_j(w_j) = G_j w_j + \frac{1}{2} (H_j + \lambda) w_j^2
$$

- $G_j = \sum_{i \in R_j} g_i$: suma de gradientes en la hoja
- $H_j = \sum_{i \in R_j} h_i$: suma de hessianos
- $\lambda$: término de regularización L2



### Evaluamos cuánto mejora la pérdida

Sustituimos el valor óptimo de nuevo en la función de pérdida para ver **cuánto mejora**:

$$
\mathcal{L}_j(w_j^*) = G_j w_j^* + \frac{1}{2}(H_j + \lambda)(w_j^*)^2
$$

Sustituyendo:

$$
\mathcal{L}_j(w_j^*) =
G_j \left(-\frac{G_j}{H_j + \lambda} \right) +
\frac{1}{2}(H_j + \lambda) \left( \frac{G_j^2}{(H_j + \lambda)^2} \right)
$$

$$
= -\frac{G_j^2}{H_j + \lambda} + \frac{1}{2} \cdot \frac{G_j^2}{H_j + \lambda}
= -\frac{1}{2} \cdot \frac{G_j^2}{H_j + \lambda}
$$

---

### Resultado

$$
\boxed{
\text{Similarity Score} = \frac{G_j^2}{H_j + \lambda}
}
$$

Este valor representa el **doble de la reducción en la pérdida** que se logra usando esa hoja con su output value óptimo.

- Es útil para comparar qué tan “buena” es una hoja antes de hacer un split.
- Cuanto mayor sea, más útil es mantener esa hoja o hacer un split basado en ella.

💡 Nota: la función de pérdida es negativa porque estamos midiendo una reducción (mejora). El **similarity score** se define como el valor positivo que indica esta ganancia potencial.

---



## 6. Gain (ganancia de un split)

Para saber si vale la pena dividir una hoja en dos, se calcula la **ganancia** del split:


Cuando se evalúa un posible split, se calcula la **ganancia esperada**:

$$
\text{Gain} = \frac{1}{2} \left( \text{Similarity}_\text{izq} + \text{Similarity}_\text{der} - \text{Similarity}_\text{padre} \right) - \gamma
$$


Si el gain es positivo, se realiza el split, $\gamma$ es un hiperparámetro



---

## 7. Predicción final

La predicción final se obtiene sumando todos los árboles:

$$
\hat{y}_i = F_0(x_i) + \sum_{t=1}^{M} f_t(x_i)
$$

Y si es clasificación, se aplica la función sigmoide para convertir a probabilidad:

$$
p_i = \frac{1}{1 + e^{-\hat{y}_i}}
$$

---

## ✅ En resumen

| Concepto            | Fórmula                                                       | Interpretación                                |
|---------------------|----------------------------------------------------------------|------------------------------------------------|
| Output value        | $w_j = -\frac{G_j}{H_j + \lambda}$                         | Valor óptimo que predice una hoja             |
| Similarity score    | $\frac{G_j^2}{H_j + \lambda}$                              | Calidad de una hoja                           |
| Gain (split)        | Ver fórmula anterior                                           | Reducción esperada en la pérdida si se divide |

---

## Derivadas para diferentes funciones de pérdida

A continuación se muestran las fórmulas del **gradiente** y el **hessiano** según el tipo de problema:

| Tipo de problema       | Función de pérdida                                 | Gradiente $g_i$                  | Hessiano $h_i$                       |
|------------------------|----------------------------------------------------|----------------------------------|--------------------------------------|
| Regresión (MSE)        | $\mathcal{L}(y_i, \hat{y}_i) = \frac{1}{2}(y_i - \hat{y}_i)^2$ | $g_i = \hat{y}_i - y_i$          | $h_i = 1$                            |
| Clasificación binaria  | $\mathcal{L}(y_i, \hat{y}_i) = -[y_i \log(p_i) + (1 - y_i)\log(1 - p_i)]$ con $p_i = \sigma(\hat{y}_i)$ | $g_i = p_i - y_i$                | $h_i = p_i(1 - p_i)$                 |

- En clasificación binaria, $\hat{y}_i$ es el **logit** y $p_i = \frac{1}{1 + e^{-\hat{y}_i}}$
- En regresión, $\hat{y}_i$ es la predicción directa del modelo

---


